# Shapiro Diagram with normalized parameters
### Cythonized, RF current in linear scale

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import *
from scipy.io import savemat
from scipy.integrate import odeint
%matplotlib inline

In [2]:
%load_ext Cython

### Resistively Shunted Model:

$\frac{d\phi}{dt}=\frac{2eR_N}{\hbar}[I_{DC}+I_{RF}\sin(2\pi f_{RF}t)-I_C\sin\phi]$

Solving $\phi(t)$, then you can get the voltage difference between the superconducting leads:

$V=\frac{\hbar}{2e}\langle\frac{d\phi}{dt}\rangle$

After Normalizing:
$I_{DC}\rightarrow I_{DC}/I_C$,

$I_{RF} \rightarrow I_{RF}/I_C$,

$ V \rightarrow I_CR_N$,

$f_0=2eI_CR_N/h$,

$f_{RF} \rightarrow f_{RF}/f_0$,

$2\pi f_0 t \rightarrow t$,

the RSJ model simply becomes:

$\frac{d\phi}{dt}=[I_{DC}+I_{RF}\sin(2\pi f_{RF}t)-\sin\phi]$

$V=\frac{\hbar}{2e}\langle\frac{d\phi}{dt}\rangle \rightarrow V=\langle\frac{d\phi}{dt}\rangle$

### Cython codes here is to speed up the simulation because python is slower than C:

In [3]:
%%cython --pgo

#To use GNU compiler gcc-10 specified in .bash_profile

cimport numpy as np
from libc.math cimport sin, pi
import numpy as np

### cdef is faster but can only be used for cython in this cell
#cpdef can be used for python outside this cell

cdef double CPR(double A, double G, double C):
    '''
        Current-phase relation
    '''
    return sin(G)+ A*sin(2*G +C*np.pi) # Normally sinusoidal

cdef double i(double t,double i_dc,double i_ac,double f_rf):
    '''
        Current applied to the junction
    '''
    return i_dc + i_ac * sin(2*pi*f_rf*t)
    
cpdef dGdt(G,double t,double i_dc,double i_ac,double f_rf,double A, double C):
    '''
        RSJ model
    '''
    der = -CPR(A,G,C) + i(t,i_dc,i_ac,f_rf)
    return der 

In [6]:
#f_array = np.array([15e9])#,13e9,7e9,5e9])
#C_array = np.array([0.16])*np.pi

f_rf=1.1

A=0.909
C=0.16 # as a unit of pi
IDC_step=0.1
IDC_array=np.arange(-5,5+IDC_step/2,IDC_step)
IRF_step=0.1
IRF_array=np.arange(0,10+IRF_step/2,IRF_step)

print("DC array size: "+str(len(IDC_array)))
print("RF array size: "+str(len(IRF_array)))

print("Driving RF frequency f_rf = "+str(f_rf))

DC array size: 101
RF array size: 101
Driving RF frequency f_rf = 1.1


In [ ]:
_name_file = "f_"+str(f_rf)+"_A"+str(np.round(A,3))+"_C"+str(np.round(C,2))+"pi_normalized"
_name_title = "f= "+str(f_rf)+"*f0), A= "+str(np.round(A,3))+"_C= "+str(np.round(C,2))+"pi_normalized"
print(_name_title)
T1=datetime.now()
print (T1)
V=np.empty([len(IRF_array),len(IDC_array)])
for i in range(0,len(IRF_array)):
    print("RF power now: "+str(i)+" of "+str(len(IRF_array))+" ,"+str(datetime.now()),end="\r")
    for j in range(0,len(IDC_array)):
        t=np.arange(0,300.01,0.01)/f_rf
        G_array= odeint(dGdt,0,t,args=(IDC_array[j],IRF_array[i],f_rf,A,C))
        V[i,j]=np.mean(np.gradient(G_array[:-1501,0]))/(0.01/f_rf)

DVDI=np.gradient(V,IDC_step,axis=1)


print ("It takes " + str(datetime.now()-T1))                
        
plt.figure()
plt.pcolormesh(IDC_array, IRF_array, DVDI, cmap = 'inferno', vmin = 0,linewidth=0,rasterized=True,shading="auto")
plt.xlabel("DC Current($I/I_C$)")
plt.ylabel("RF Current ($I_RF/I_C$)")
plt.colorbar(label = "DV/DI")
plt.title(_name_title)
#plt.savefig("DVDI_"+_name_file+".pdf")
plt.show()
                
plt.figure()
plt.pcolormesh(IDC_array, IRF_array, V , cmap = 'coolwarm',linewidth=0,rasterized=True,shading="auto")
plt.xlabel("DC Current($I/I_C$)")
plt.ylabel("RF Current ($I_RF/I_C$)")
plt.colorbar(label = "$V/I_CR_N$")
plt.title(_name_title)
#plt.savefig("V_"+_name_file+".pdf")
plt.show()

plt.figure()
plt.plot(IDC_array,V[1,:])#/(np.pi*hbar*f/Qe))
plt.show()
plt.figure()
plt.plot(IDC_array,DVDI[1,:])
plt.show()
#savemat("data"+_name_file+'.mat',mdict={'IDC':IDC_array,'IRF':IRF_array,'PRF':PRF_array,'A':A, 'w_rf':w_rf,'C':C,'Vmatrix':V,'DVDI':DVDI})
#print('file saved')

f= 1.1*f0), A= 0.909_C= 0.16pi_normalized
2020-09-24 17:40:49.578067
